In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [2]:
path = "./dino_nms.json"
dataframe = pd.read_json(path)

In [3]:
dataframe['image_id'] = ["test/"+str(i).zfill(4)+".jpg" for i in dataframe["image_id"]]
bbox = np.array([i for i in dataframe["bbox"]])
bbox[:, 2], bbox[:,3] = bbox[:, 0] + bbox[:, 2], bbox[:, 1] + bbox[:, 3]
bbox = [str(i[0]) + " " +str(i[1]) +" " +str(i[2]) +" " +str(i[3]) for i in bbox]

In [4]:
temp = []
for i in range(len(dataframe["score"])):
    temp.append(str(dataframe["score"][i]) +" "+ bbox[i])
dataframe["PredictionString"] = temp

In [5]:
new_df = pd.DataFrame(columns=["PredictionString","image_id"])

# loop through each image in the original table
for img in dataframe["image_id"].unique():
    # select only the rows for the current image
    img_df = dataframe[dataframe["image_id"] == img]
    
    # create the PredictionString for the current image
    pred_str = ""
    for _, row in img_df.iterrows():
        bbox_str = " ".join(str(x) for x in row["bbox"])
        score_str = str(row["score"])
        cat_str = str(row["category_id"])
        pred_str += f"{cat_str} {score_str} {bbox_str} "
    
    # add the new row to the new dataframe
    new_row = {"image_id": img, "PredictionString": pred_str}
    new_df = pd.concat([new_df, pd.DataFrame(new_row, index=[0])], ignore_index=True)

In [21]:
new_df.to_csv("./dino_nms.csv")